In [3]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
#import cv2
from skimage.transform import resize

from tensorflow.keras.models import model_from_json


Using TensorFlow backend.


In [11]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    
    # Display some key metadata
    #print('Patient Name:', ds.get('PatientName', 'N/A'))
    #print('Modality:', ds.get('Modality', 'N/A'))
    #print('Study Date:', ds.get('StudyDate', 'N/A'))
    print('Label:', ds.get('StudyDescription', 'N/A'))
    #print('Series Description:', ds.get('SeriesDescription', 'N/A'))
    #print('Image Comments:', ds.get('ImageComments', 'N/A'))
    #print('Protocol Name:', ds.get('ProtocolName', 'N/A'))
    
    img = ds.pixel_array
    return img

# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # Resize the image to the target size
    # print(img.shape)
    # print(img.ndim)
    resized_image = resize(img, img_size, anti_aliasing=True)
    #resized_image = cv2.resize(img, img_size)    
    # Convert the image to float32 for normalization
    normalized_image = resized_image.astype(np.float32)
    normalized_image = (normalized_image - img_mean) / img_std
    return normalized_image
        

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # Load the JSON file
    with open(model_path, "r") as json_file:
        mode_json = json_file.read()
    model = model_from_json(mode_json)
    model.load_weights(weight_path)
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prediction = model.predict(img, verbose = True)
    return prediction 

In [13]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path =  '/workspace/home/best/VGG16_1.0_n512_l_r_0.001_l0.0001_dr_0.1_l2_0.0001_d0.5_b128.json'
weight_path = '/workspace/home/best/VGG16_1.0_n512_l_r_0.001_l0.0001_dr_0.1_l2_0.0001_d0.5_b128_loss.h5'


IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0# loads the mean image value they used during training preprocessing
img_std = 0# loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.4142516
categories = ['NO_PNEUMONIA','PNEUMONIA'] 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
    img_mean = np.mean(img)
    img_std = np.std(img)
            
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    pred_t = (pred[0,1]>thresh).astype(int)
    print(f'prediction: {categories[pred_t]}')


Load file test1.dcm ...
Label: No Finding
1/1 [==============================] - 1s 1s/sample
prediction: NO_PNEUMONIA
Load file test2.dcm ...
Label: Cardiomegaly
1/1 [==============================] - 1s 1s/sample
prediction: NO_PNEUMONIA
Load file test3.dcm ...
Label: Effusion
1/1 [==============================] - 1s 956ms/sample
prediction: NO_PNEUMONIA
Load file test4.dcm ...
Label: No Finding
1/1 [==============================] - 1s 964ms/sample
prediction: NO_PNEUMONIA
Load file test5.dcm ...
Label: No Finding
1/1 [==============================] - 1s 932ms/sample
prediction: NO_PNEUMONIA
Load file test6.dcm ...
Label: No Finding
1/1 [==============================] - 1s 913ms/sample
prediction: NO_PNEUMONIA
